In [4]:
from miditok import MMM, MuMIDI, TokenizerConfig, Octuple, REMI
from miditok.pytorch_data import DatasetTok, DataCollator
from pathlib import Path
from symusic import Score
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import os

### Tokenizer 선언

In [12]:
TOKENIZER_PARAMS = {
    "pitch_range": (0, 127),
    "num_velocities": 127,
    # "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "max_bar_embedding" : 3000,
    "use_chords": True,
    "use_tempos": True,
    "use_programs": True,
    "num_tempos": 211,  # number of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
config = TokenizerConfig(**TOKENIZER_PARAMS)
TOKENIZER_NAME = REMI
tokenizer = TOKENIZER_NAME(config)
len(tokenizer)

3769

### 음악이 저장된 폴더

In [13]:
midi_paths = list(Path("jazz-midi-366-songs").glob("*.mid"))
print(midi_paths)

[PosixPath('jazz-midi-366-songs/279_Samba_do_Aviao.mid'), PosixPath('jazz-midi-366-songs/033_bayou.mid'), PosixPath('jazz-midi-366-songs/047_bbossaje.mid'), PosixPath('jazz-midi-366-songs/035_beacon.mid'), PosixPath('jazz-midi-366-songs/218_misty.mid'), PosixPath('jazz-midi-366-songs/134_GrooveYard.mid'), PosixPath('jazz-midi-366-songs/079_CarnHallBounce.mid'), PosixPath('jazz-midi-366-songs/207_carnival.mid'), PosixPath('jazz-midi-366-songs/291_SingSing.mid'), PosixPath('jazz-midi-366-songs/316_straight.mid'), PosixPath('jazz-midi-366-songs/004_2ndtime.mid'), PosixPath('jazz-midi-366-songs/034_bbblues.mid'), PosixPath('jazz-midi-366-songs/195_LittlePixieG.mid'), PosixPath('jazz-midi-366-songs/113_doxy.mid'), PosixPath('jazz-midi-366-songs/048_BLBOSSA.mid'), PosixPath('jazz-midi-366-songs/157_Idkwhtime.mid'), PosixPath('jazz-midi-366-songs/012_ALICEINW.mid'), PosixPath('jazz-midi-366-songs/158_gotrhytm.mid'), PosixPath('jazz-midi-366-songs/246_OpFunk.mid'), PosixPath('jazz-midi-366-son

In [22]:

### 노래가 저장되어있는 폴더 명
dir_path ='jazz-midi-366-songs'
### 저장할 폴더
CHUNK_PATH = './split_bar'

## 몇 바를 기준으로 나눌 것인가
bar_num = 3

for song in tqdm(os.listdir(dir_path)[:1]):
    if not os.path.exists(f'{CHUNK_PATH}/{song[:-4]}'):
            os.makedirs(f'{CHUNK_PATH}/{song[:-4]}')
    print(song)
    tokens = tokenizer(f'{dir_path}/{song}')
    li=[]
    tempo =0
    for ids, token in enumerate(tokens.tokens):
        if ids == 0:
            li.append(token)
            continue
        if tokens.events[ids].type_ =='Tempo':
            tempo = token
            continue
        if tokens.events[ids].type_ =='Bar':
            li.append(tempo)
            current_bar = int(token[4:])
            if current_bar % bar_num == 0 :
                 
                tokenizer(li).dump_midi(f'{CHUNK_PATH}/{song[:-4]}/{current_bar}.mid')
                li=[]


            li.append(token)
            continue
        li.append(token)
    li.append(tempo)
    tokenizer(li).dump_midi(f'{CHUNK_PATH}/{song[:-4]}/{current_bar+1}.mid')
    print(current_bar+bar_num)


100%|██████████| 1/1 [00:00<00:00, 16.02it/s]

279_Samba_do_Aviao.mid
44


In [18]:
midi_paths = list(Path("jazz-midi-366-songs").glob("*.mid"))
print(midi_paths[0])
tokens = tokenizer(midi_paths[0])
tokens

jazz-midi-366-songs/279_Samba_do_Aviao.mid


TokSequence(tokens=['Bar_0', 'Position_0', 'Tempo_70.0', 'Position_2', 'Program_-1', 'PitchDrum_69', 'Velocity_40', 'Duration_0.1.8', 'Position_6', 'Program_-1', 'PitchDrum_69', 'Velocity_40', 'Duration_0.1.8', 'Position_10', 'Program_-1', 'PitchDrum_69', 'Velocity_40', 'Duration_0.1.8', 'Position_14', 'Program_-1', 'PitchDrum_69', 'Velocity_40', 'Duration_0.1.8', 'Position_16', 'Program_0', 'Pitch_56', 'Velocity_48', 'Duration_1.0.8', 'Program_0', 'Pitch_59', 'Velocity_50', 'Duration_1.0.8', 'Program_0', 'Pitch_68', 'Velocity_74', 'Duration_1.1.8', 'Program_0', 'Pitch_52', 'Velocity_56', 'Duration_1.0.8', 'Program_0', 'Pitch_49', 'Velocity_45', 'Duration_1.0.8', 'Program_35', 'Pitch_25', 'Velocity_85', 'Duration_1.2.8', 'Program_-1', 'PitchDrum_69', 'Velocity_48', 'Duration_0.1.8', 'Position_18', 'Program_-1', 'PitchDrum_69', 'Velocity_40', 'Duration_0.1.8', 'Position_20', 'Program_-1', 'PitchDrum_69', 'Velocity_48', 'Duration_0.1.8', 'Position_22', 'Program_-1', 'PitchDrum_69', 'Velo

In [9]:
path_dir ='jazz-midi-366-songs'

for song in os.listdir(path_dir):
    print(song[:-4])


279_Samba_do_Aviao
033_bayou
047_bbossaje
035_beacon
218_misty
134_GrooveYard
079_CarnHallBounce
207_carnival
291_SingSing
316_straight
004_2ndtime
034_bbblues
195_LittlePixieG
113_doxy
048_BLBOSSA
157_Idkwhtime
012_ALICEINW
158_gotrhytm
246_OpFunk
112_DstMeT
132_gsteps
188_kicker2
183_joyed
323_atrain2
326_tempusfugit
172_intheevening
292_Sodanco
268_RhyANing
050_BlueMonk
097_CottonG
285_Senor2
269_riofunk
162_I_Wont_Dance_R
273_romwargm
202_Lulu
171_in_love_with_night
174_INVITATION
040_bdazhed
203_LushLife
294_softly
003_20thcenturystomp
332_TheManILove
196_Lovehere
065_Bop
346_Wave3
258_perissc
036_beale
365_zanzibar
098_criss-cross
324_TakinAC
142_hayBurner
015_AllOfMe2
329_ipanema
264_polkadot
355_WoodynYou
100_Datdere2
136_groovn
244_one4hellen
205_mvoyage
163_If_bell
243_ontheotherhand
266_prelude
140_HaveUheard
275_rndmngt
336_tp
296_someday
046_Blknrcs
007_AFoggyD
229_naima95
286_serialnumber
095_conalma
351_whisper
295_solar
044_billiesb
038_Berniestune
228_my1aonly
363_youd